DataSet: [https://www.kaggle.com/ruthgn/wine-quality-data-set-red-white-wine](https://www.kaggle.com/ruthgn/wine-quality-data-set-red-white-wine)

Целевая переменная - "quality"

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from scipy import stats
from scipy.special import boxcox1p
from sklearn.model_selection import train_test_split

In [ ]:
path = 'wine-quality-white-and-red.csv'
data = pd.read_csv(path)
data_copy = data.copy()

In [ ]:
data.head(10)

# Гистограммы

Выбросов в районе нуля не видно. Вероятно, все данные определены.

Целевая переменная - категориальная, параметры - непрерывные.

На многих графиках присутствуют пустотные места, например, на графике `citric acid` больше 0.85. Вероятно, там есть выбросы.

In [ ]:
data.hist(figsize=(20, 20), bins=30);

# Пытаемся найти выбросы по гистограммам

In [ ]:
data = data_copy.copy()

## Остаточный сахар
Не смотря на то, что значения остаточного сахара меньше 1 редки, оставляем, т.к. их не мало, и на выбросы они не похожи.
Значения больше 30 считаем выбросами.

In [ ]:
_, ax = plt.subplots(1, 3, figsize=(20, 4))
data[data['residual sugar'] < 1.5]['residual sugar'].hist(bins=8, ax=ax[0])
data[data['residual sugar'] > 18]['residual sugar'].hist(bins=15, ax=ax[1])
data = data.drop(data[data['residual sugar'] > 30].index)
data['residual sugar'].hist(bins=30, ax=ax[2]);

## Свободный диоксид серы

Считаем выбросами значения больше 200.

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))
data[data['free sulfur dioxide'] > 70]['free sulfur dioxide'].hist(bins=30, ax=ax[0])
data = data.drop(data[data['free sulfur dioxide'] > 200].index)
data['free sulfur dioxide'].hist(bins=30, ax=ax[1]);

## Лимонная кислота

Считаем выбросами значения больше 0.9.

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))
data[data['citric acid'] > 0.65]['citric acid'].hist(bins=30, ax=ax[0])
data = data.drop(data[data['citric acid'] > 0.9].index)
data['citric acid'].hist(bins=30, ax=ax[1]);

## Хлориды

Думаю, можно посчитать значение около 0.6 выбросом, с остальными не понятно.


In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))
data[data['chlorides'] > 0.2]['chlorides'].hist(bins=30, ax=ax[0])
data = data.drop(data[data['chlorides'] > 0.55].index)
data['chlorides'].hist(bins=30, ax=ax[1]);

## Сульфаты

Считаем значения около 2 выбросами.

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(15, 4))
data[data['sulphates'] > 1]['sulphates'].hist(bins=30, ax=ax[0])
data = data.drop(data[data['sulphates'] > 1.8].index)
data['sulphates'].hist(bins=30, ax=ax[1]);

## СКО Хлоридов

Точки `(quality=7, chlorides~0.35)` и `(quality=4, chlorides~0.3)` похожи на выбросы.

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(20, 8))
sns.boxplot(data=data, x='quality', y='chlorides', ax=ax[0])
data = data.drop(data[(data['quality'] == 7) & (data['chlorides'] > 0.3)].index)
data = data.drop(data[(data['quality'] == 4) & (data['chlorides'] > 0.25)].index)
sns.boxplot(data=data, x='quality', y='chlorides', ax=ax[1]);

# Пропущенные значения

Все значения определены.

In [ ]:
nulls_counts = data.isnull().sum()
nulls_counts

# Корреляция

In [ ]:
corr_table = data.corr()
_, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(corr_table[(corr_table <= -0.4) | (corr_table >= 0.4)],
            square=True, vmin=-1, vmax=1, cmap='PRGn', ax=ax,
            annot=True,
            annot_kws={'size': 16});

Посмотрим на 2 наиболее коррелированные пары величин:
- Свободный диоксид серы и Общий диоксид серы коррелированы по понятной причине - Общий включает в себя Свободный.
- Плотность и Алкоголь

Добавим поле для связанного диоксида серы

In [ ]:
data['bound sulfur dioxide'] = data['total sulfur dioxide'] - data['free sulfur dioxide']
data = data[['type', 'fixed acidity', 'volatile acidity', 'citric acid',
             'residual sugar', 'chlorides', 'free sulfur dioxide', 'bound sulfur dioxide',
             'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']]

Из графика зависимости алкоголя и плотности видно, что есть сильно выраженная нижняя граница алкоголя при низкой плотности.
Еще при `(alcohol~15, density~0.997)` похоже есть выброс.

In [ ]:
_, ax = plt.subplots(1, 2, figsize=(12, 4))
sns.scatterplot(data=data, x='density', y='alcohol', ax=ax[0])
data = data.drop(data[(data['alcohol'] > 14) & (data['density'] > 0.996)].index)
sns.scatterplot(data=data, x='density', y='alcohol', ax=ax[1]);

# Гистограммы после чистки

In [ ]:
data.hist(figsize=(20, 20), bins=30);

# Смещение

Смещения всех значений

In [ ]:
numeric_index = data.dtypes[data.dtypes != 'object'].index

all_skewness = data[numeric_index]\
    .apply(lambda x: stats.skew(x))\
    .sort_values(ascending=False)\
    .rename('skewness')
all_skewness = pd.DataFrame(all_skewness)
all_skewness

Наиболее смещенные значения.
Определим коэффициенты лямбда для преобразования.

In [ ]:
skewness = all_skewness[abs(all_skewness['skewness']) > 0.75].copy()
skewness['lambda'] = [-20, -1, -3, -2, 0.1, 0.5]
skewness

Преобразуем

In [ ]:
data_after_transform = data.copy()
for i in skewness.index:
    data_after_transform[i] = boxcox1p(data[i], skewness.at[i, 'lambda'])

Смещенности после преобразования

In [ ]:
data_after_transform[numeric_index]\
    .apply(lambda x: x.skew())\
    .sort_values(ascending=False)

# Гистограммы после преобразования

In [ ]:
data_after_transform.hist(figsize=(20, 20), bins=30);

# Попарные зависимости

In [ ]:
pd.plotting.scatter_matrix(data_after_transform, figsize=(20, 20));

# Разделение на тренировочные и тестовые данные

In [ ]:
train, test = train_test_split(data_after_transform, test_size=0.3)
train.shape, test.shape